### AHP Function

In [87]:
import numpy as np
import pandas as pd


# Function untuk menghitung bobot (weights)
def ahp_weights(matrix_comparism):
    
    # matrix_comparism = np.array(matrix_comparism)
    n = matrix_comparism.shape[0]

    # Menghitung nilai geometris dari matriks perbandingan
    M_I = np.prod(matrix_comparism, axis=1)
    W_bar_I = np.power(M_I, 1/n)

    # Normalisasi bobot relatif agar jumlahnya 1
    W_I = W_bar_I / np.sum(W_bar_I)
    
    return W_I, W_bar_I

Dict_RI = {
    1: 0.00, 
    2: 0.00, 
    3: 0.58, 
    4: 0.90, 
    5: 1.12, 
    6: 1.24, 
    7: 1.32, 
    8: 1.41, 
    9: 1.45,
    10: 1.49,
    11: 1.51,
    12: 1.48,
    13: 1.56,
    14: 1.57,
    15: 1.58,
}

# Function untuk menghitung konsistensi
def consistency(matrix_comparism, W_I):
    n = matrix_comparism.shape[0]

    # Menghitung lambda max
    AW= np.dot(matrix_comparism, W_I)
    lambda_max = np.sum(AW / (n*W_I))
    
    # Menghitung CI
    CI = (lambda_max - n) / (n - 1)

    # Mengambil RI sesuai dengan n
    RI = Dict_RI[n]

    # Menghitung CR
    CR = CI / RI 

    return lambda_max, CI, CR, RI, n

def ahp_rumus(matrix_comparism, labels):
    # Hitung bobot
    W_I, W_bar_I = ahp_weights(matrix_comparism)
    
    # Hitung konsistensi
    lambda_max, CI, CR, RI, n = consistency(matrix_comparism, W_I)
    
    # Buat DataFrame untuk menampilkan hasil matriks perbandingan, geometris, dan bobot
    df = pd.DataFrame(matrix_comparism, 
                      index=labels, 
                      columns=labels,
                    )
    df['Geometris'] = W_bar_I
    df['Weight'] = W_I
    
    # Cetak output sesuai format
 
    print("Comparism Matrix")
    print(df.round(3))
    print(f'\nJumlah Bobot = {round(W_I.sum())}')
    print(f'\nLambda_max= {lambda_max:.4f}')
    print(f'RI= {n} : {RI}')
    print(f'CI= {CI:.4f}')
    print(f'CR= {CR:.4}')

    # Evaluasi konsistensi
    if CR < 0.1:
        print("\nBobot konsisten")
    else:
        print("Bobot Tidak Konsisten")

    return W_I

AHP parameter utama

In [88]:

iklim_topografi = 3
iklim_tanah = 1
topografi_tanah = 1/5

matrix_comparism = np.array([
    [1, iklim_topografi, iklim_tanah],
    [1/iklim_topografi, 1, topografi_tanah],
    [1/iklim_tanah, 1/topografi_tanah, 1]
])

# Label untuk parameter
labels = ['iklim', 'topografi', 'tanah']

# Memanggil fungsi AHP untuk menampilkan hasil
# ahp_rumus(matrix_comparism, labels)

# Memanggil fungsi AHP dan mendapatkan hasil bobot (W_I)
W_I = ahp_rumus(matrix_comparism, labels)

# Membuat DataFrame hanya dengan label dan bobot
df_output_utama = pd.DataFrame({
    'Label': labels,
    'Weight': W_I
})

# Menyimpan DataFrame ke file Excel dengan nama yang berbeda
# df_output.to_excel('./dataset/bobot_ahp/parameter_utama.xlsx', index=False)


Comparism Matrix
           iklim  topografi  tanah  Geometris  Weight
iklim      1.000        3.0    1.0      1.442   0.405
topografi  0.333        1.0    0.2      0.405   0.114
tanah      1.000        5.0    1.0      1.710   0.481

Jumlah Bobot = 1

Lambda_max= 3.0291
RI= 3 : 0.58
CI= 0.0145
CR= 0.02505

Bobot konsisten


AHP Sub parameter Iklim

In [89]:

#suhu [A]; curah_hujan [B], kelembapan_udara [C], penyinaran [D]
A_B = 1/3
A_C = 5
A_D = 1
B_C = 7
B_D = 3
C_D = 1/5

A, B, C, D = 1,1,1,1  

# Matriks perbandingan
matrix_comparism = np.array([
    [A, A_B, A_C, A_D],
    [1/A_B, B, B_C, B_D],
    [1/A_C, 1/B_C, C, C_D],
    [1/A_D, 1/B_D, 1/C_D, D]
])

# Label untuk parameter
labels = ['suhu', 'curah_hujan', 'kelembapan_udara', 'penyinaran']

# Memanggil fungsi AHP untuk menampilkan hasil
# ahp_rumus(matrix_comparism, labels)

# Memanggil fungsi AHP dan mendapatkan hasil bobot (W_I)
W_I = ahp_rumus(matrix_comparism, labels)

# Membuat DataFrame hanya dengan label dan bobot
df_output_iklim = pd.DataFrame({
    'Label': labels,
    'Weight': W_I
})

# Menyimpan DataFrame ke file Excel dengan nama yang berbeda
# df_output.to_excel('./dataset/bobot_ahp/parameter_iklim.xlsx', index=False)

Comparism Matrix
                  suhu  curah_hujan  kelembapan_udara  penyinaran  Geometris  \
suhu               1.0        0.333               5.0         1.0      1.136   
curah_hujan        3.0        1.000               7.0         3.0      2.817   
kelembapan_udara   0.2        0.143               1.0         0.2      0.275   
penyinaran         1.0        0.333               5.0         1.0      1.136   

                  Weight  
suhu               0.212  
curah_hujan        0.525  
kelembapan_udara   0.051  
penyinaran         0.212  

Jumlah Bobot = 1

Lambda_max= 4.0732
RI= 4 : 0.9
CI= 0.0244
CR= 0.0271

Bobot konsisten


AHP sub parameter topografi

In [90]:
# Ketinggian [A], Kemiringan [B], Drainase [C]
A,B,C = 1,1,1

A_B = 5
A_C = 1
B_C = 1/3

matrix_comparism = np.array([
    [A, A_B, A_C],
    [1/A_B, B, B_C],
    [1/A_C, 1/B_C, C]
])

# Label untuk parameter
labels = ['ketinggian', 'kemiringan', 'drainase']

# # Memanggil fungsi AHP untuk menampilkan hasil
# ahp_rumus(matrix_comparism, labels)

# Memanggil fungsi AHP dan mendapatkan hasil bobot (W_I)
W_I = ahp_rumus(matrix_comparism, labels)

# Membuat DataFrame hanya dengan label dan bobot
df_output_topo = pd.DataFrame({
    'Label': labels,
    'Weight': W_I
})

# Menyimpan DataFrame ke file Excel dengan nama yang berbeda
# df_output.to_excel('./dataset/bobot_ahp/parameter_topografi.xlsx', index=False)

Comparism Matrix
            ketinggian  kemiringan  drainase  Geometris  Weight
ketinggian         1.0         5.0     1.000      1.710   0.481
kemiringan         0.2         1.0     0.333      0.405   0.114
drainase           1.0         3.0     1.000      1.442   0.405

Jumlah Bobot = 1

Lambda_max= 3.0291
RI= 3 : 0.58
CI= 0.0145
CR= 0.02505

Bobot konsisten


AHP sub parameter Tanah

In [91]:
# ph [A], Kelembapan Tanah [B], Jenis Tanah [C], Tekstur Tanah [D]
A,B,C,D = 1,1,1,1

A_B = 3
A_C = 1
A_D = 5
B_C = 1/5
B_D = 1
C_D = 5

matrix_comparism = np.array([
    [A, A_B, A_C, A_D],
    [1/A_B, B, B_C, B_D],
    [1/A_C, 1/B_C, C, C_D],
    [1/A_D, 1/B_D, 1/C_D, D]
])

labels = ['ph', 'kelembapan_tanah', 'jenis_tanah', 'tekstur_tanah']

# ahp_rumus(matrix_comparism, labels)

# Memanggil fungsi AHP dan mendapatkan hasil bobot (W_I)
W_I = ahp_rumus(matrix_comparism, labels)

# Membuat DataFrame hanya dengan label dan bobot
df_output_tanah = pd.DataFrame({
    'Label': labels,
    'Weight': W_I
})

# Menyimpan DataFrame ke file Excel dengan nama yang berbeda
# df_output.to_excel('./dataset/bobot_ahp/parameter_tanah.xlsx', index=False)

Comparism Matrix
                     ph  kelembapan_tanah  jenis_tanah  tekstur_tanah  \
ph                1.000               3.0          1.0            5.0   
kelembapan_tanah  0.333               1.0          0.2            1.0   
jenis_tanah       1.000               5.0          1.0            5.0   
tekstur_tanah     0.200               1.0          0.2            1.0   

                  Geometris  Weight  
ph                    1.968   0.381  
kelembapan_tanah      0.508   0.098  
jenis_tanah           2.236   0.433  
tekstur_tanah         0.447   0.087  

Jumlah Bobot = 1

Lambda_max= 4.0327
RI= 4 : 0.9
CI= 0.0109
CR= 0.01212

Bobot konsisten


AHP sub parameter jenis tanah

In [92]:
#asosiasi latosol cokelat dan regosol kelabu [A]
#  kompleks regosol dan litosol [B], 
# kompleks litosol, mediteran, renzina [C], 
# mediteran [D]
# regosol [E]

A_B = 2
A_C = 3
A_D = 5
A_E = 3
B_C = 2
B_D = 5
B_E = 3
C_D = 3
C_E = 2
D_E = 1/3

A, B, C, D, E = 1,1,1,1,1  

# Matriks 4x4 yang simetris untuk perbandingan keempat kriteria
matrix_comparism = np.array([
    [A, A_B, A_C, A_D, A_E],
    [1/A_B, B, B_C, B_D, B_E],
    [1/A_C, 1/B_C, C, C_D, C_E],
    [1/A_D, 1/B_D, 1/C_D, D, D_E],
    [1/A_E, 1/B_E, 1/C_E, 1/D_E, E]
])

# Label untuk parameter
labels = ['asosiasi latosol cokelat dan regosol kelabu', 'kompleks regosol dan litosol', 'kompleks litosol, mediteran, renzina', 'mediteran', 'regosol']

# ahp_rumus(matrix_comparism, labels)

# Memanggil fungsi AHP dan mendapatkan hasil bobot (W_I)
W_I = ahp_rumus(matrix_comparism, labels)

# Membuat DataFrame hanya dengan label dan bobot
df_output_jenis_tanah = pd.DataFrame({
    'Label': labels,
    'Weight': W_I
})

# Menyimpan DataFrame ke file Excel dengan nama yang berbeda
# df_output.to_excel('./dataset/bobot_ahp/bobot_jenis_tanah.xlsx', index=False)

Comparism Matrix
                                             asosiasi latosol cokelat dan regosol kelabu  \
asosiasi latosol cokelat dan regosol kelabu                                        1.000   
kompleks regosol dan litosol                                                       0.500   
kompleks litosol, mediteran, renzina                                               0.333   
mediteran                                                                          0.200   
regosol                                                                            0.333   

                                             kompleks regosol dan litosol  \
asosiasi latosol cokelat dan regosol kelabu                         2.000   
kompleks regosol dan litosol                                        1.000   
kompleks litosol, mediteran, renzina                                0.500   
mediteran                                                           0.200   
regosol                                      

### PROFILE MATCHING

Define Dataset

In [93]:
import pandas as pd

#import dataset varietas
df_varietas_iklim = pd.read_excel('./dataset/split_data/varietas_iklim.xlsx')
df_varietas_topo = pd.read_excel('./dataset/split_data/varietas_topografi.xlsx')
df_varietas_tanah = pd.read_excel('./dataset/split_data/varietas_tanah.xlsx')

#import dataset lahan
df_lahan_iklim = pd.read_excel('./dataset/split_data/lahan_iklim.xlsx')
df_lahan_topo = pd.read_excel('./dataset/split_data/lahan_topografi.xlsx')
df_lahan_tanah = pd.read_excel('./dataset/split_data/lahan_tanah.xlsx')

define_varietas = "V3"

Interpolasi Iklim

In [94]:
# Function utama untuk interpolasi
def interpolasi(x, min_val, max_val):
    if min_val <= x <= max_val:
        return 5
    elif 0 <= x < min_val:
        return 1 + ((x - 0) / (min_val - 0)) * (5 - 1)
    elif max_val < x <= (min_val + max_val):
        return 5 + ((x - max_val) / ((min_val + max_val) - max_val)) * (1 - 5)
    else:
        return 1

# Function untuk menghitung interpolasi berdasarkan varietas tertentu (misalnya, hanya V1)
def iklim(row, jenis_varietas):
    # Memastikan jenis varietas ada di dataset
    if jenis_varietas not in df_varietas_iklim['jenis_varietas'].values:
        raise ValueError(f"Varietas {jenis_varietas} tidak ditemukan di dataset.")

    # Mengambil data varietas berdasarkan jenis_varietas
    var_data = df_varietas_iklim[df_varietas_iklim['jenis_varietas'] == jenis_varietas].iloc[0]
    
    # Menghitung skor interpolasi untuk masing-masing parameter
    hasil = {
        'suhu': interpolasi(row['suhu_mean'], var_data['suhu_min'], var_data['suhu_max']),
        'curah_hujan': interpolasi(row['curah_hujan'], var_data['curah_hujan_min'], var_data['curah_hujan_max']),
        'kelembapan_udara': interpolasi(row['kelembapan_udara'], var_data['kelembapan_udara_min'], var_data['kelembapan_udara_max']),
        'penyinaran': interpolasi(row['penyinaran'], var_data['penyinaran_min'], var_data['penyinaran_max'])
    }
    
    return hasil

#Penggunaan 
def hitung_skor(row):
    return iklim(row, define_varietas)

lahan_scores = df_lahan_iklim.apply(hitung_skor, axis=1)

# Menampilkan DataFrame hasil dengan nama lahan dan skor interpolasi 
interpolasi_iklim = pd.concat([df_lahan_iklim['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)

print(f"Hasil Skor Interpolasi Iklim untuk Varietas {define_varietas}:")
interpolasi_iklim


Hasil Skor Interpolasi Iklim untuk Varietas V3:


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran
0,Bakung,5.0,5.0000,5.0,5.0
1,Binangun,5.0,5.0000,5.0,5.0
2,Doko,5.0,1.0000,5.0,5.0
3,Gandusari,5.0,1.0000,5.0,5.0
4,Garum,5.0,1.0000,5.0,5.0
5,Kademangan,5.0,3.5296,5.0,5.0
6,Kanigoro,5.0,5.0000,5.0,5.0
7,Kesamben,5.0,1.9368,5.0,5.0
8,Nglegok,5.0,1.0000,5.0,5.0
9,Panggungrejo,5.0,5.0000,5.0,5.0


Interpolasi topografi

In [95]:

def interpolasi_ketinggian(x, min_val,max_val):
    if  min_val <= x <= max_val:
        return 5
    elif 0 <= x < min_val:
        return 1 + ((x - 0) / (min_val - 0)) * (5 - 1)
    elif max_val < x <= (min_val + max_val):
        return 5 + ((x - max_val) / ((min_val + max_val) - max_val)) * (1 - 5)
    else:
        return 1

def interpolasi_kemiringan(x,min_val, max_val):
    if min_val <= x <= max_val:
        return 5
    elif max_val < x <= 80 :
        return 5 + ((x - max_val) / (80  - max_val)) * (1 - 5)
    else: 
        return 1

def interpolasi_drainase(x, value):
    if x == value:
        return 5
    else:
        return 1
    
def topografi(row, jenis_varietas):
    var_data = df_varietas_topo[df_varietas_topo['jenis_varietas'] == jenis_varietas].iloc[0]

    hasil = {
        'ketinggian': interpolasi_ketinggian(row['ketinggian'], var_data['ketinggian_min'], var_data['ketinggian_max']),
        'kemiringan' : interpolasi_kemiringan(row['kemiringan'], var_data['kemiringan_min'], var_data['kemiringan_max']),
        'drainase': interpolasi_drainase(row['drainase'], var_data['drainase'])
    }

    return hasil

#Penggunaan 
def hitung_skor(row):
    return topografi(row, define_varietas)

lahan_scores = df_lahan_topo.apply(hitung_skor, axis=1)

# Menampilkan DataFrame hasil dengan nama lahan dan skor interpolasi 
interpolasi_topo = pd.concat([df_lahan_topo['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)

print(f"Hasil Skor Interpolasi Topografi untuk Varietas: {define_varietas}")
interpolasi_topo



Hasil Skor Interpolasi Topografi untuk Varietas: V3


,nama_lahan,ketinggian,kemiringan,drainase
0,Bakung,5.0,5.000000,5.0
1,Binangun,5.0,5.000000,5.0
2,Doko,1.0,5.000000,5.0
3,Gandusari,1.0,5.000000,5.0
4,Garum,5.0,5.000000,5.0
5,Kademangan,5.0,5.000000,1.0
6,Kanigoro,5.0,5.000000,5.0
7,Kesamben,5.0,5.000000,5.0
8,Nglegok,1.0,5.000000,5.0
9,Panggungrejo,1.0,3.857143,5.0


Interpolasi Tanah

In [96]:
df_jenis_tanah = df_output_jenis_tanah

# Fungsi interpolasi untuk variabel numerik
def interpolasi(x, min_val, max_val):
    if min_val <= x <= max_val:
        return 5
    elif 0 <= x < min_val:
        return 1 + ((x - 0) / (min_val - 0)) * (5 - 1)
    elif max_val < x <= (min_val+max_val):
        return 5 + ((x - max_val) / ((min_val + max_val) - max_val)) * (1 - 5)
    else:
        return 1

# Interpolasi untuk tekstur
def interpolasi_tekstur(x, value):
    if x in ['lempung', 'liat', 'berpasir', 'lempung berpasir']:
        return 5
    else:
        return 1

def interpolasi_jenis_tanah(x):
    if -1 <= x <= 0:
        return 5 + ((x - 0) / (-1 - 0)) * (4 - 5)
    elif 0 <= x <= 1:
        return 5 + ((x - 0) / (1 - 0)) * (4.5 - 5)

# Mengubah jenis tanah menjadi bobot berdasarkan data bobot jenis tanah
def ubah_jenis_tanah_ke_bobot(df_lahan_tanah, df_bobot_tanah):
    bobot_dict = pd.Series(df_bobot_tanah.Weight.values, index=df_bobot_tanah.Label).to_dict()
    df_lahan_tanah['jenis_tanah_bobot'] = df_lahan_tanah['jenis_tanah'].map(bobot_dict)
    return df_lahan_tanah

df_lahan_tanah = ubah_jenis_tanah_ke_bobot(df_lahan_tanah, df_jenis_tanah)

# Ubah varietas tanah ke bobot
def ubah_varietas_tanah_ke_bobot(df_varietas_tanah_tanah, df_bobot_tanah):
    bobot_dict = pd.Series(df_bobot_tanah.Weight.values, index=df_bobot_tanah.Label).to_dict()
    df_varietas_tanah_tanah['jenis_tanah_bobot'] = df_varietas_tanah_tanah['jenis_tanah'].map(bobot_dict)
    return df_varietas_tanah_tanah

df_varietas_tanah = ubah_varietas_tanah_ke_bobot(df_varietas_tanah, df_jenis_tanah)

# Hitung selisih bobot antara lahan dan varietas
def hitung_selisih_bobot(df_lahan_tanah, df_varietas_tanah, varietas):
    varietas_bobot = df_varietas_tanah[df_varietas_tanah['jenis_varietas'] == varietas].iloc[0]['jenis_tanah_bobot']
    df_lahan_tanah['selisih_bobot'] = df_lahan_tanah['jenis_tanah_bobot'] - varietas_bobot
    return df_lahan_tanah


# Hitung skor tanah dengan menggunakan data selisih bobot
def skor_tanah(row, jenis_varietas):
    var_data = df_varietas_tanah[df_varietas_tanah['jenis_varietas'] == jenis_varietas].iloc[0]
    
    hasil = {
        'ph': interpolasi(row['ph'], var_data['ph_min'], var_data['ph_max']),
        'kelembapan_tanah': interpolasi(row['kelembapan_tanah'], var_data['kelembapan_tanah_min'], var_data['kelembapan_tanah_max']),
        'jenis_tanah': interpolasi_jenis_tanah(row['selisih_bobot']),
        'tekstur_tanah': interpolasi_tekstur(row['tekstur_tanah'], var_data['tekstur_tanah'])
    }
    return hasil

# Penggunaan
df_lahan_tanah = hitung_selisih_bobot(df_lahan_tanah, df_varietas_tanah, define_varietas)
lahan_scores = df_lahan_tanah.apply(lambda row: skor_tanah(row, define_varietas), axis=1)

# Menampilkan DataFrame hasil dengan nama lahan dan skor interpolasi 
interpolasi_tanah = pd.concat([df_lahan_tanah['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)

print(f"Hasil Skor Interpolasi Tanah untuk Varietas: {define_varietas}:")
interpolasi_tanah


Hasil Skor Interpolasi Tanah untuk Varietas: V3:


,nama_lahan,ph,kelembapan_tanah,jenis_tanah,tekstur_tanah
0,Bakung,5.0,4.626667,4.884360,1.0
1,Binangun,5.0,4.626667,4.777935,1.0
2,Doko,5.0,3.640000,4.940413,5.0
3,Gandusari,5.0,3.640000,5.000000,5.0
4,Garum,5.0,3.640000,4.835906,5.0
5,Kademangan,5.0,4.160000,4.884360,5.0
6,Kanigoro,5.0,4.626667,4.835906,5.0
7,Kesamben,5.0,4.986667,4.777935,5.0
8,Nglegok,5.0,3.640000,5.000000,5.0
9,Panggungrejo,5.0,4.626667,4.884360,1.0


## Nilai Kriteria

Nilai Kriteria Iklim

In [97]:
#bobot ahp sub parameter iklim
bobot_iklim = df_output_iklim
bobot_iklim_dict = bobot_iklim.set_index('Label')['Weight'].to_dict()
bobot_iklim_dict

{'suhu': 0.21179576152333418,
 'curah_hujan': 0.5251582768815123,
 'kelembapan_udara': 0.05125020007181938,
 'penyinaran': 0.21179576152333418}

In [98]:
nk_iklim = interpolasi_iklim.copy()

# Melakukan perkalian setiap kolom dengan bobot yang sesuai
for kolom in ['suhu', 'curah_hujan', 'kelembapan_udara', 'penyinaran']:
    nk_iklim[kolom] = nk_iklim[kolom] * bobot_iklim_dict[kolom]

nk_iklim['nk_iklim'] = nk_iklim[['suhu', 'curah_hujan', 'kelembapan_udara', 'penyinaran']].sum(axis=1)

nk_iklim


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran,nk_iklim
0,Bakung,1.058979,2.625791,0.256251,1.058979,5.000000
1,Binangun,1.058979,2.625791,0.256251,1.058979,5.000000
2,Doko,1.058979,0.525158,0.256251,1.058979,2.899367
3,Gandusari,1.058979,0.525158,0.256251,1.058979,2.899367
4,Garum,1.058979,0.525158,0.256251,1.058979,2.899367
5,Kademangan,1.058979,1.853599,0.256251,1.058979,4.227807
6,Kanigoro,1.058979,2.625791,0.256251,1.058979,5.000000
7,Kesamben,1.058979,1.017127,0.256251,1.058979,3.391335
8,Nglegok,1.058979,0.525158,0.256251,1.058979,2.899367
9,Panggungrejo,1.058979,2.625791,0.256251,1.058979,5.000000


nilai kriteria topografi

In [99]:
#bobot ahp topografi
bobot_topo = df_output_topo
bobot_topo_dict = bobot_topo.set_index('Label')['Weight'].to_dict()
bobot_topo_dict

{'ketinggian': 0.48064008518129187,
 'kemiringan': 0.11397236672339138,
 'drainase': 0.4053875480953168}

In [100]:
nk_topografi = interpolasi_topo.copy()

for kolom in ['ketinggian','kemiringan','drainase']:
    nk_topografi[kolom] = nk_topografi[kolom] * bobot_topo_dict[kolom]

# Menambahkan kolom 'total_skor' yang berisi jumlah dari kolom yang telah dikalikan bobot
nk_topografi['nk_topo'] = nk_topografi[['ketinggian','kemiringan','drainase']].sum(axis=1)

nk_topografi

,nama_lahan,ketinggian,kemiringan,drainase,nk_topo
0,Bakung,2.40320,0.569862,2.026938,5.000000
1,Binangun,2.40320,0.569862,2.026938,5.000000
2,Doko,0.48064,0.569862,2.026938,3.077440
3,Gandusari,0.48064,0.569862,2.026938,3.077440
4,Garum,2.40320,0.569862,2.026938,5.000000
5,Kademangan,2.40320,0.569862,0.405388,3.378450
6,Kanigoro,2.40320,0.569862,2.026938,5.000000
7,Kesamben,2.40320,0.569862,2.026938,5.000000
8,Nglegok,0.48064,0.569862,2.026938,3.077440
9,Panggungrejo,0.48064,0.439608,2.026938,2.947186


Nilai Kriteria Tanah

In [101]:
#bobot ahp sub parameter tanah
bobot_tanah = df_output_tanah
bobot_tanah_dict = bobot_tanah.set_index('Label')['Weight'].to_dict()
bobot_tanah_dict

{'ph': 0.38143740520354763,
 'kelembapan_tanah': 0.09848671453159402,
 'jenis_tanah': 0.4333965668873818,
 'tekstur_tanah': 0.08667931337747636}

In [102]:
# Variabel untuk menyimpan hasil perkalian dengan bobot
nk_tanah = interpolasi_tanah.copy()

# Melakukan perkalian setiap kolom dengan bobot yang sesuai
for kolom in ['ph', 'kelembapan_tanah', 'jenis_tanah', 'tekstur_tanah']:
    nk_tanah[kolom] = nk_tanah[kolom] * bobot_tanah_dict[kolom]

# Menambahkan kolom 'total_skor' yang berisi jumlah dari kolom yang telah dikalikan bobot
nk_tanah['nk_tanah'] = nk_tanah[['ph', 'kelembapan_tanah', 'jenis_tanah', 'tekstur_tanah']].sum(axis=1)

nk_tanah


,nama_lahan,ph,kelembapan_tanah,jenis_tanah,tekstur_tanah,nk_tanah
0,Bakung,1.907187,0.455665,2.116865,0.086679,4.566396
1,Binangun,1.907187,0.455665,2.070741,0.086679,4.520272
2,Doko,1.907187,0.358492,2.141158,0.433397,4.840233
3,Gandusari,1.907187,0.358492,2.166983,0.433397,4.866058
4,Garum,1.907187,0.358492,2.095865,0.433397,4.794940
5,Kademangan,1.907187,0.409705,2.116865,0.433397,4.867153
6,Kanigoro,1.907187,0.455665,2.095865,0.433397,4.892114
7,Kesamben,1.907187,0.491120,2.070741,0.433397,4.902445
8,Nglegok,1.907187,0.358492,2.166983,0.433397,4.866058
9,Panggungrejo,1.907187,0.455665,2.116865,0.086679,4.566396


### Rangking

In [103]:
#bobot parameter utama
bobot_utama = pd.read_excel('./dataset/bobot_ahp/parameter_utama.xlsx')
bobot_utama_dict = bobot_utama.set_index('Label')['Weight'].to_dict()
bobot_utama_dict

{'iklim': 0.4053875480953168,
 'topografi': 0.1139723667233914,
 'tanah': 0.4806400851812919}

In [104]:
# Mengalikan kolom 'skor nk iklim' dengan bobot 'iklim'
total_skor_iklim = nk_iklim.copy()
total_skor_iklim['total_skor_iklim'] = total_skor_iklim['nk_iklim'] * bobot_utama_dict['iklim']
total_skor_iklim


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran,nk_iklim,total_skor_iklim
0,Bakung,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
1,Binangun,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
2,Doko,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
3,Gandusari,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
4,Garum,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
5,Kademangan,1.058979,1.853599,0.256251,1.058979,4.227807,1.713900
6,Kanigoro,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
7,Kesamben,1.058979,1.017127,0.256251,1.058979,3.391335,1.374805
8,Nglegok,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
9,Panggungrejo,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938


In [105]:
# Mengalikan kolom 'skor nk topo' dengan bobot 'topografi'
total_skor_topo = nk_topografi.copy()
total_skor_topo['total_skor_topo'] = total_skor_topo['nk_topo'] * bobot_utama_dict['topografi']
total_skor_topo


,nama_lahan,ketinggian,kemiringan,drainase,nk_topo,total_skor_topo
0,Bakung,2.40320,0.569862,2.026938,5.000000,0.569862
1,Binangun,2.40320,0.569862,2.026938,5.000000,0.569862
2,Doko,0.48064,0.569862,2.026938,3.077440,0.350743
3,Gandusari,0.48064,0.569862,2.026938,3.077440,0.350743
4,Garum,2.40320,0.569862,2.026938,5.000000,0.569862
5,Kademangan,2.40320,0.569862,0.405388,3.378450,0.385050
6,Kanigoro,2.40320,0.569862,2.026938,5.000000,0.569862
7,Kesamben,2.40320,0.569862,2.026938,5.000000,0.569862
8,Nglegok,0.48064,0.569862,2.026938,3.077440,0.350743
9,Panggungrejo,0.48064,0.439608,2.026938,2.947186,0.335898


In [106]:
# Mengalikan kolom 'nk tanah' dengan bobot 'tanah'
total_skor_tanah = nk_tanah.copy()
total_skor_tanah['total_skor_tanah'] = total_skor_tanah['nk_tanah'] * bobot_utama_dict['tanah']
total_skor_tanah


,nama_lahan,ph,kelembapan_tanah,jenis_tanah,tekstur_tanah,nk_tanah,total_skor_tanah
0,Bakung,1.907187,0.455665,2.116865,0.086679,4.566396,2.194793
1,Binangun,1.907187,0.455665,2.070741,0.086679,4.520272,2.172624
2,Doko,1.907187,0.358492,2.141158,0.433397,4.840233,2.326410
3,Gandusari,1.907187,0.358492,2.166983,0.433397,4.866058,2.338823
4,Garum,1.907187,0.358492,2.095865,0.433397,4.794940,2.304641
5,Kademangan,1.907187,0.409705,2.116865,0.433397,4.867153,2.339349
6,Kanigoro,1.907187,0.455665,2.095865,0.433397,4.892114,2.351346
7,Kesamben,1.907187,0.491120,2.070741,0.433397,4.902445,2.356311
8,Nglegok,1.907187,0.358492,2.166983,0.433397,4.866058,2.338823
9,Panggungrejo,1.907187,0.455665,2.116865,0.086679,4.566396,2.194793


perangkingan

In [107]:
# Menggabungkan ketiga DataFrame berdasarkan 'nama_lahan'
combined_scores = total_skor_iklim.merge(total_skor_tanah, on='nama_lahan').merge(total_skor_topo, on='nama_lahan')

# Menambahkan kolom total skor dari penjumlahan 'total_skor_iklim', 'total_skor_tanah', dan 'total_skor_topo'
combined_scores['total_skor'] = (
    combined_scores['total_skor_iklim'] + 
    combined_scores['total_skor_tanah'] + 
    combined_scores['total_skor_topo']
)

# Mengurutkan berdasarkan 'total_skor' secara descending dan 'nama_lahan' secara ascending
combined_scores = combined_scores.sort_values(by=['total_skor', 'nama_lahan'], ascending=[False, True])

# Menentukan peringkat unik
combined_scores['rank'] = range(1, len(combined_scores) + 1)

# Menampilkan hasil yang diurutkan
print(f'Rangking lahan pertanian varietas: {define_varietas}')
rangking = combined_scores[['nama_lahan', 'total_skor', 'rank']]
rangking


Rangking lahan pertanian varietas: V3


,nama_lahan,total_skor,rank
11,Sanankulon,4.982328,1
6,Kanigoro,4.948146,2
12,Selopuro,4.948146,3
15,Sutojayan,4.948146,4
0,Bakung,4.791593,5
1,Binangun,4.769424,6
13,Selorejo,4.606201,7
9,Panggungrejo,4.557629,8
14,Srengat,4.462388,9
17,Udanawu,4.462388,10
